# TSP Project

### Generic Helpers 

In [54]:
def load_tsp_data(city_names, city_dist):
    """
    Reads text files of city names and city distances and forms lists so they can be referred to by the algorithms
    
    :param city_names: text file containing city names
    :param city_dist: text file containing distances between cities
    """
    # Load city names
    with open(city_names) as read_file:
        city_names_list = [line.strip() for line in read_file.readlines()]
    # Load distance matrix
    distance_matrix = []
    with open(city_dist) as read_file:
        for line in read_file:
            city_str_list =  line.split()
            one_city_list = [float(distance) for distance in city_str_list]
            distance_matrix.append(one_city_list)
    return city_names_list, distance_matrix

In [55]:
def distance(route, distances):
    """
    Find the total distance between all cities in a route

    :param route: the full route of cities
    """
    distance = 0

    # loops through cities and adds distances
    for index in range(len(route)-1):
        distance += distances[route[index]][route[index+1]]

    # add distance to return to first city
    distance += distances[route[-1]][route[0]]
    
    return distance

### Backtracking

##### Psuedocode

```
helper function to parse txt
    read distance file and parse into lists for each city
    read name file and create list of names
    returns lists

helper function to calculate distance
    for city in the route and back to the first city
        add distances
    return distances

helper function tsp_recursive with parameter partial_route, remaining cities
    global routes
    if list of cities is empty
        return partial
    else 
        for city in remaining_cities
            partial copy
            append remaining city

            remaining_cities copy
            remove remaining city
            tsp_recursive(partial copy, remaining copy)

create list variable for possible routes
tsp_backtracking function with name.txt and distance.txt as parameters
    make list variable global

    call parse txt helper
    make list form 0-6 for the 7 cities

    for second city in len of list of cities
        for the last city from the second city to len of list of cities
            one possible route is equal [0,second_city, last city]  
            make a copy of the city list and remove these 3 to get the remaining cities

            call tsp_recursive with the partial route and remaing
    
    for each route in list of routes
        make a route distances list by calling the distance helper
    
    find the shortest distance from route distances
    find the best_route that matches that distance

    attach route names to best route
    print results
```

In [56]:
routes = []
def tsp_recursive(partial_route, remaining_cities):
    """
    Recursive portion of the backtracking algorithm

    :param thirty_cities_names: text file containing the names of seven cities
    :param thirty_cities_dist: text file containing the distances between all seven cities
    """
    global routes

    # if none remaining end recursive
    if len(remaining_cities) == 0:
        routes.append(partial_route)
    else:
        # for each remaing city, insert it in the second to last position and remove it from remaing
        for city in remaining_cities:
            partial_route_copy = partial_route.copy()
            partial_route_copy.insert(-1, city)

            remaining_cities_copy = remaining_cities.copy()
            remaining_cities_copy.remove(city)
            tsp_recursive(partial_route_copy, remaining_cities_copy)

def tsp_backtracking(seven_cities_names, seven_cities_dist):
    """
    Backtracking algorithm to find the shortest route that touches all  cities and ends at the start

    :param seven_cities_names: text file containing the names of seven cities
    :param seven_cities_dist: text file containing the distances between all seven cities
    """
    global routes
    city_names, distances = load_tsp_data(seven_cities_names,  seven_cities_dist)
    city_list = [i for i in range(0,7)]
    route_distances = []

    # creates starting cases of of routes with second and last city
    for second_city in range(1,len(city_list)):
        for last_city in range(second_city+1, len(city_list)):
            partial_route = [0, second_city, last_city]
            
            # removes all cities in partial_route from remaining
            remaining = city_list.copy()
            for city in partial_route:
                remaining.remove(city)

            tsp_recursive(partial_route, remaining)
    
    # makes a list of distances the same length as routes
    for route in routes:
        route_distances.append(distance(route, distances))
    
    # finds the best route from routes
    shortest_distance = min(route_distances)
    route_index = route_distances.index(shortest_distance)
    best_route = routes[route_index]

    # attach route names and print results
    route_names = [city_names[i] for i in best_route]
    print('Best Backtracking Route:', " -> ".join(route_names),"->", route_names[0])
    print("Total Distance:", shortest_distance)

In [57]:
routes = []
tsp_backtracking('seven_cities_names.txt', 'seven_cities_dist.txt')

Best Backtracking Route: Alpha -> Epsilon -> Gamma -> Delta -> Zeta -> Beta -> Eta -> Alpha
Total Distance: 106.4


### Evolutionary

##### Psuedocode

```
helper function to parse txt
    read distance file and parse into lists for each city
    read name file and create list of names
    returns lists

helper function to calculate distance
    for city in the route and back to the first city
        add distances
    return distances

tsp_evolutionary function with the thirty city names and thirty city distances as parameters
    call parse text helper function
    best_route initialized as list 0-29
    call distance helper on best route and save as shortest_distance
    stagnation_count
    stagnation_routes empty list
    stagnation_distances empty list
    
    for generation from 1 to 10000
        if stagnation occurs 3 times
            leave loop
        if no improvement occurs
            append current best_route to stagnation_routes
            append current shortest_distance to stagnation_distance
            make 5 random mutations
            increment stagnation count
    
        save best_route and shortest_distance as parent

        for 1000 offspring:
            make copy of parent as offspring

            switch two random indexes in offspring
            see what the new distance is

            if offspring beats shortest_distance
                change best_route and shortest_route to the offspring values

    find the shortest_distance in the stagnation_distances
    find the accompanying route in the stagnation_routes

    attach route names to the best list
    print results
```

In [ ]:
def tsp_evolutionary(thirty_cities_names, thirty_cities_dist):
    """
    Evolutionary algorithm to find the shortest route that touches all thirty cities and ends at the start

    :param thirty_cities_names: text file containing the names of thirty cities
    :param thirty_cities_dist: text file containing the distances between all thirty cities
    """
    from random import sample 
    city_names, distances = load_tsp_data(thirty_cities_names, thirty_cities_dist)
    best_route = [num for num in range(0,30)]
    shortest_distance = distance(best_route, distances)
    parent_distance = 0 
    stagnation_count = 0
    stagnation_routes = []
    stagnation_distances = []

    # loop through 10000 generations
    for generation in range(1,10000):
        #leave loop if 3 stagnations occur
        if stagnation_count == 3:
            break
        # checks if distance changed from previous parent
        if shortest_distance == parent_distance:
            print(f"Max value reached of {parent_distance} for stagnation {stagnation_count+1}.")
            stagnation_routes.append(best_route)
            stagnation_distances.append(shortest_distance)
            # 5 random mutations
            for i in range(5):
                (randint1, randint2) = sample(range(0,30),2)
                city1 = best_route[randint1]
                city2 = best_route[randint2]
            
                best_route[randint1] = city2
                best_route[randint2] = city1

            shortest_distance = distance(best_route, distances)
            stagnation_count += 1
        
        # set parent values
        parent = best_route
        parent_distance = shortest_distance

        # run through 1000 offspring
        for offspring in range(1000):
            offspring = parent.copy()

            # switch two random indexes
            (randint1, randint2) = sample(range(0,30),2)
            city1 = parent[randint1]
            city2 = parent[randint2]
        
            offspring[randint1] = city2
            offspring[randint2] = city1
            
            offspring_distance = distance(offspring, distances)
            
            # if improvement is made in offspring, save new route
            if offspring_distance < shortest_distance:
                best_route = offspring
                shortest_distance = offspring_distance

    # find the best_route out of the stagnations
    shortest_distance = min(stagnation_distances)
    route_index = stagnation_distances.index(shortest_distance)
    best_route = stagnation_routes[route_index]

    # attach route names and print results
    route_names = [city_names[i] for i in best_route]
    print('Best Evolutionary Route:', " -> ".join(route_names),"->", route_names[0])
    print("Total Distance:", shortest_distance)

In [44]:
tsp_evolutionary("thirty_cities_names.txt", "thirty_cities_dist.txt")

Max value reached of 600.0 for stagnation 1.
Max value reached of 600.0 for stagnation 2.
Max value reached of 600.0 for stagnation 3.
Best Evolutionary Route: Capetown -> Cairo -> Istanbul -> Rome -> Berlin -> New York -> Bombay -> Rio de Janeiro -> Guam -> Tokyo -> Shanghai -> Juneau -> Melbourne -> Paris -> Azores -> Montreal -> New Orleans -> Chicago -> Baghdad -> Mexico City -> Panama City -> Manila -> Buenos Aires -> Santiago -> San Francisco -> Seattle -> Moscow -> Honolulu -> Sydney -> London -> Capetown
Total Distance: 600.0


### Greedy

##### Psuedocode

```
BEGIN tsp_greedy(name_file, distance_file) 

# loading cities
    city_names=read_lines_from(thirty_cities_names.txt)
    distance_matrix=read_2D_floats_from(thirty_cities_dist.txt)
    n=number of cities in city_names
    best_route=EMPTY
    best_distance=INFINITY

# trying each city from starting point
    for start_city from 0 to n-1 do
        visited=empty list
        visited.add(start_city)
        current_city=start_city
        total_distance=0

# building the route
        while number of visited cities<n Do

    # finding next unvisted city
            nearist_city=None
            nearist_distcance=infinite 
            FOR next_city FROM 0 TO n-1 DO
                IF next_city NOT IN visited THEN
                    distance=distance_matrix[current_city][next_city]
                    IF distance<nearest_distance THEN
                        nearest_distance=distance
                        nearest_city=next_city

    # Moves to nearest city
            visited.add(nearest_city)
            total_distance=total_distance + nearest_distance
            current_city=nearest_city
        ENDWHILE loop

# returns to starting city 
        total_distance=total_distance + distance_matrix[current_city][start_city]
        visited.add(start_city)  # completes the loop

# checks for best route 
        IF total_distance<best_distance THEN
            best_distance=total_distance
            best_route=copy_of(visited)
    ENDFOR loop

# prints results
    PRINT "Best greedy route:", convert_indices_to_names(best_route, city_names)
    PRINT "Total distance:", best_distance

END tsp_greedy
```

In [45]:
def tsp_greedy(thirty_cities_names, thirty_cities_dist):
    """
    Greedy algorithm to find the shortest route that touches all thirty cities and ends at the starting city

    :param thirty_cities_names: text file containing the names of thirty cities
    :param thirty_cities_dist: text file containing the distances between all thirty cities
    """
    # uses the helper function to load up the data
    city_names, distance_matrix = load_tsp_data(thirty_cities_names, thirty_cities_dist)
    n = len(city_names)
    best_route = []
    best_distance = float('inf') # float('inf') is a place holder infinity value so any distance will be smaller
    # trys each city from starting point to find the smallest distance 
    for start_city in range(n):
        visited = [start_city]
        current_city = start_city
        total_distance = 0
        # starts trying to build a route
        while len(visited) < n:
            # finding the next unvisted city
            nearest_city = None
            nearest_distance = float('inf')
            for next_city in range(n):
                if next_city not in visited:
                    distance = distance_matrix[current_city][next_city]
                    if distance < nearest_distance:
                        nearest_distance = distance
                        nearest_city = next_city
            # Moves to nearest city 
            visited.append(nearest_city)
            total_distance += nearest_distance
            current_city = nearest_city
        # returns to starting city
        total_distance += distance_matrix[current_city][start_city]
        visited.append(start_city)
        # checks for best route
        if total_distance < best_distance:
            best_distance = total_distance
            best_route = visited
    # prints the results
    route_names = [city_names[i] for i in best_route]
    print("Best Greedy Route:", " -> ".join(route_names))
    print("Total Distance:", round(best_distance, 2))


In [38]:
tsp_greedy("thirty_cities_names.txt", "thirty_cities_dist.txt")

Best Greedy Route: Azores -> London -> Paris -> Berlin -> Rome -> Istanbul -> Cairo -> Baghdad -> Moscow -> Bombay -> Shanghai -> Tokyo -> Guam -> Manila -> Melbourne -> Sydney -> Honolulu -> San Francisco -> Seattle -> Juneau -> Chicago -> New York -> Montreal -> New Orleans -> Mexico City -> Panama City -> Santiago -> Buenos Aires -> Rio de Janeiro -> Capetown -> Azores
Total Distance: 527.0
